In [12]:
# Environment sanity check
import sys
import torch
import transformers
import pydantic
import json
import platform

print("=== Environment Sanity Check ===")
print(f"Python version: {sys.version}")
print(f"Platform: {platform.platform()}")
print(f"PyTorch version: {torch.__version__}")
print(f"Transformers version: {transformers.__version__}")
print(f"Pydantic version: {pydantic.__version__}")
print(f"CUDA available: {torch.cuda.is_available()}")

=== Environment Sanity Check ===
Python version: 3.10.19 | packaged by Anaconda, Inc. | (main, Oct 21 2025, 16:41:31) [MSC v.1929 64 bit (AMD64)]
Platform: Windows-10-10.0.26200-SP0
PyTorch version: 2.5.1
Transformers version: 5.1.0
Pydantic version: 2.12.5
CUDA available: True


In [13]:
from transformers import AutoTokenizer, AutoModelForCausalLM

MODEL_NAME = "microsoft/Phi-3-mini-4k-instruct"  # or your chosen local model

print(f"Loading tokenizer: {MODEL_NAME}")
tokenizer = AutoTokenizer.from_pretrained(MODEL_NAME)

print(f"Loading model: {MODEL_NAME}")
model = AutoModelForCausalLM.from_pretrained(
    MODEL_NAME,
    torch_dtype=torch.float32,
    device_map="cpu",
)

print("Model and tokenizer loaded successfully.")

Loading tokenizer: microsoft/Phi-3-mini-4k-instruct
Loading model: microsoft/Phi-3-mini-4k-instruct


Loading weights: 100%|██████████| 195/195 [00:16<00:00, 11.90it/s, Materializing param=model.norm.weight]                              


Model and tokenizer loaded successfully.


In [14]:
text = "Test input: patient has systolic BP > 180 mmHg."
tokens = tokenizer(text, return_tensors="pt")
decoded = tokenizer.decode(tokens["input_ids"][0])

print("Original:", text)
print("Decoded :", decoded)

Original: Test input: patient has systolic BP > 180 mmHg.
Decoded : Test input: patient has systolic BP > 180 mmHg.


In [15]:
from pydantic import BaseModel, Field, ValidationError
from typing import List

class Citation(BaseModel):
    chunk_id: str
    line_start: int
    line_end: int

class ExtractedFact(BaseModel):
    fact_type: str
    text: str
    citations: List[Citation]

sample = {
    "fact_type": "threshold",
    "text": "Systolic blood pressure ≥ 180 mmHg requires urgent evaluation.",
    "citations": [
        {"chunk_id": "guideline_01", "line_start": 12, "line_end": 14}
    ],
}

try:
    fact = ExtractedFact(**sample)
    print("Pydantic validation: PASSED")
    print(json.dumps(fact.model_dump(), indent=2))
except ValidationError as e:
    print("Pydantic validation: FAILED")
    print(e)

Pydantic validation: PASSED
{
  "fact_type": "threshold",
  "text": "Systolic blood pressure \u2265 180 mmHg requires urgent evaluation.",
  "citations": [
    {
      "chunk_id": "guideline_01",
      "line_start": 12,
      "line_end": 14
    }
  ]
}


In [16]:
prompt = "Extract explicitly stated clinical thresholds only."

inputs = tokenizer(prompt, return_tensors="pt")
with torch.no_grad():
    outputs = model.generate(
        **inputs,
        max_new_tokens=30,
        do_sample=False,
    )

print(tokenizer.decode(outputs[0], skip_special_tokens=True))

Extract explicitly stated clinical thresholds only.

Document:

Clinical Trial: Efficacy of Drug X in Treating Chronic Pain

Background:
